In [98]:
import lyricsgenius
import json
import pandas as pd
import nltk
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.tokenize import word_tokenize
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk import word_tokenize, sent_tokenize 
import re

In [6]:
#Checking whether installed module works correctly, setting Eminem as the author
genius = lyricsgenius.Genius("jHJB8vqrbfVDIGM1KrETFx6s7EG9SGN3RGcmr63z7Gjgz8dYY4LizTL9rGupTAl1")
artist = genius.search_artist("Eminem", max_songs=100, sort="title")
genius.remove_section_headers = True
genius.verbose = False
print('')

Searching for songs by Eminem...

Song 1: "12 Days of Diss-Mas"
Song 2: "1-833-2GET-REV (REVIVAL Voicemail)"
Song 3: "1996 Underground Freestyle"
Song 4: "1997 Freestyle Live at Wetlands, NYC"
Song 5: "1997 Rap Olympics"
Song 6: "1999"
Song 7: "1999 Tim Westwood Freestyle"
Song 8: "2004 Tim Westwood Freestyle"
Song 9: "2012 Something From Nothing: Art of Rap Freestyle"
Song 10: "2.0 Boys"
Song 11: "25 To Life"
Song 12: "313"
Song 13: "3 a.m."
Song 14: "3 Verses"
Song 15: "4 Verses"
Song 16: "50 Ways"
Song 17: "8 Mile"
Song 18: "8 Mile: 313 Cypher (Chin Tiki Girls)"
Song 19: "8 Mile Background Music"
Song 20: "8 Mile: B-Rabbit vs Papa Doc"
Song 21: "8 Mile: B-Rabbit vs Supa Emcee"
Song 22: "8 Mile: Cheddar Bob Freestyle"
Song 23: "8 Mile: D’Phuzion vs B-Rabbit"
Song 24: "8 Mile: Lily’s Lullaby"
Song 25: "8 Mile: Lotto vs B-Rabbit"
Song 26: "8 Mile: Lyckety-Splyt vs B-Rabbit"
Song 27: "8 Mile: Marv Won vs B-Rabbit"
Song 28: "8 Mile: Maurice Grant vs B-Rabbit"
Song 29: "8 Mile: Sweet Home

In [7]:
artist.save_lyrics()

Wrote `Lyrics_Eminem.json`


In [28]:
lyrics = 'Lyrics_Eminem.json'
with open(lyrics) as eminem_lyrics:
    messyvocab = json.load(eminem_lyrics)
dict_keys = messyvocab.keys()
print(dict_keys)

dict_keys(['alternate_names', 'api_path', 'description', 'facebook_name', 'followers_count', 'header_image_url', 'id', 'image_url', 'instagram_name', 'is_meme_verified', 'is_verified', 'name', 'translation_artist', 'twitter_name', 'url', 'current_user_metadata', 'iq', 'description_annotation', 'user', 'songs'])


In [57]:
vocab = messyvocab.get('songs')
lyrics_vocab = []
for i in range(0,len(vocab)):
    index = vocab[i]
    song = index.get('lyrics')
    lyrics_vocab.append(song)
lyricsdf = pd.DataFrame(lyrics_vocab, columns=['Song lyrics'])
lyricsdf.head(3)

,Song lyrics
0,"1. ""Hit 'Em Up"" by 2Pac (Ft. Outlawz)\n2. ""Dis..."
1,[Automated Voice]\nThank you for your interest...
2,Any rapper saying those kind of rhymes\nIn thi...


In [59]:
lyricsdf = lyricsdf.dropna()
print(len(lyricsdf))

99


In [60]:
stopsymbols = ['[', ',', ']', "'", '(', ')', 'verse', 'intro', 'refrain', '1','1.','2','2.','3','3.','4','4.','5','5.','6','6.','7','7.','8','8.','9','9.','10','10.', ':', '!', '—', '"']
for i in range(0,len(lyricsdf)):
    lyric = lyricsdf.iloc[i,0]
    for j in stopsymbols:
        cleanlyric=lyric.replace(j, '')
        lyricsdf.iloc[i,0] =cleanlyric

In [96]:
nltk.download('punkt')
lyricsdf['Song lyrics'] = lyricsdf['Song lyrics'].str.lower()
tokenized_lyr = lyricsdf.apply(lambda row: nltk.tokenize.word_tokenize(row['Song lyrics']), axis=1)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Forest_Gump\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [99]:
train_data, padded_sents = padded_everygram_pipeline(2, tokenized_lyr)

In [101]:
from nltk.lm import KneserNeyInterpolated as KNI

In [102]:
model = KNI(2)

In [103]:
model.fit(train_data, padded_sents)
print(model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 7795 items>


In [106]:
print(model.vocab.lookup(tokenized_lyr[2]))

('any', 'rapper', 'saying', 'those', 'kind', 'of', 'rhymes', 'in', 'this', 'day', 'and', 'age', 'in', 'this', 'period', 'of', 'time', 'tryna', 'battle', 'eminem', 'is', 'worse', 'than', 'david', 'starr', 'tryna', 'battle', 'proof', ',', 'eye-ku', ',', 'bflat', 'and', 'bizarre', '(', 'unintelligible', ')', 'that', "'s", 'why', 'you', 'beg', 'to', 'differ', 'me', ',', 'you', 'ai', "n't", 'got', 'no', 'style', 'plus', 'you', 'lack', 'delivery', ',', 'not', 'to', 'brag', 'i', 'do', "n't", 'need', 'to', 'boast', 'look', ',', 'my', 'face', 'is', 'pale', ',', 'but', 'you', 'look', 'like', 'you', 'seen', 'a', 'ghost', '(', 'unintelligible', ')', 'you', 'could', "n't", 'slip', 'up', 'in', 'the', 'zone', ',', 'if', 'i', 'put', 'banana', 'peels', 'around', 'this', 'fucking', 'microphone', 'so', 'get', 'a', 'bulldozer', ',', 'start', 'from', 'bottom', 'to', 'top', 'and', 'tear', 'this', 'building', 'down', "'cause", 'that', "'s", 'the', 'only', 'way', 'you', 'wrecking', 'shop')


In [107]:
print(model.generate(20, random_seed=7))

['fate', 'that', 'many', 'calibers', 'here', ',', 'and', 'just', 'a', 'head-butt', 'so', 'big', 'now', 'so', 'chill', '!', 'i', 'was', 'never', 'leave']


In [115]:
from nltk.util import ngrams
from nltk.lm import NgramCounter
text_unigrams = [ngrams(sent, 1) for sent in tokenized_lyr]
text_bigrams = [ngrams(sent, 2) for sent in tokenized_lyr]
text_trigrams = [ngrams(sent, 3) for sent in tokenized_lyr]

In [108]:
from nltk.lm.smoothing import KneserNey

In [123]:
smooth = KneserNey( _,NgramCounter)

In [84]:
sent_tokenize = lambda x: re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', x)
toktok = ToktokTokenizer()
word_tokenize = word_tokenize = toktok.tokenize

In [90]:
tokenized_text = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in sent_tokenize(list_lyr)]
print(len(tokenized_text))

1


In [173]:
# Tokenize the text.
tokenized_text = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in sent_tokenize(all_lyrics)]
print(len(tokenized_text))

7


In [174]:
joined_lyrics = []
[ joined_lyrics.extend(el) for el in tokenized_text] 

print(len(joined_lyrics))

5168


In [175]:
joined_lyrics

['[',
 'intro',
 ']',
 '*eminem',
 'pukes',
 'into',
 'the',
 'toilet*',
 '*eminem',
 'hocks',
 'and',
 'spits',
 'into',
 'the',
 'toilet*',
 '*toilet',
 'flush*',
 'there',
 'i',
 'go',
 ',',
 'thinking',
 'of',
 'you',
 'again',
 '[',
 'hook',
 ']',
 'you',
 'don',
 "'",
 't',
 'know',
 'how',
 'sick',
 'you',
 'make',
 'me',
 'you',
 'make',
 'me',
 'fucking',
 'sick',
 'to',
 'my',
 'stomach',
 'every',
 'time',
 'i',
 'think',
 'of',
 'you',
 'i',
 'puke',
 'you',
 'must',
 'just',
 'not',
 'kno-o-o-o-o-o-ow',
 'you',
 'may',
 'not',
 'think',
 'you',
 'do',
 ',',
 'but',
 'you',
 'do',
 'every',
 'time',
 'i',
 'think',
 'of',
 'you',
 'i',
 'puke',
 '[',
 'verse',
 '1',
 ']',
 'i',
 'was',
 'gonna',
 'take',
 'the',
 'time',
 'to',
 'sit',
 'down',
 'and',
 'write',
 'you',
 'a',
 'little',
 'poem',
 'but',
 'off',
 'of',
 'the',
 'dome',
 'would',
 'probably',
 'be',
 'a',
 'little',
 'more',
 'more',
 'suitable',
 'for',
 'this',
 'type',
 'of',
 'song',
 ',',
 'whoa',
 'i',


In [184]:
stopwords=['[', ',', ']', "'", '(', ')', 'verse', 'intro', 'refrain', '1', '2', '3', ':', '!', '—', '"']
for word in joined_lyrics:
    if word in stopwords:
        joined_lyrics.remove(word)

In [188]:
list(bigrams(joined_lyrics))

[('*eminem', 'pukes'),
 ('pukes', 'into'),
 ('into', 'the'),
 ('the', 'toilet*'),
 ('toilet*', '*eminem'),
 ('*eminem', 'hocks'),
 ('hocks', 'and'),
 ('and', 'spits'),
 ('spits', 'into'),
 ('into', 'the'),
 ('the', 'toilet*'),
 ('toilet*', '*toilet'),
 ('*toilet', 'flush*'),
 ('flush*', 'there'),
 ('there', 'i'),
 ('i', 'go'),
 ('go', 'thinking'),
 ('thinking', 'of'),
 ('of', 'you'),
 ('you', 'again'),
 ('again', 'hook'),
 ('hook', 'you'),
 ('you', 'don'),
 ('don', 't'),
 ('t', 'know'),
 ('know', 'how'),
 ('how', 'sick'),
 ('sick', 'you'),
 ('you', 'make'),
 ('make', 'me'),
 ('me', 'you'),
 ('you', 'make'),
 ('make', 'me'),
 ('me', 'fucking'),
 ('fucking', 'sick'),
 ('sick', 'to'),
 ('to', 'my'),
 ('my', 'stomach'),
 ('stomach', 'every'),
 ('every', 'time'),
 ('time', 'i'),
 ('i', 'think'),
 ('think', 'of'),
 ('of', 'you'),
 ('you', 'i'),
 ('i', 'puke'),
 ('puke', 'you'),
 ('you', 'must'),
 ('must', 'just'),
 ('just', 'not'),
 ('not', 'kno-o-o-o-o-o-ow'),
 ('kno-o-o-o-o-o-ow', 'you'),


Heavily based on:
    https://www.kaggle.com/alvations/n-gram-language-model-with-nltk